# RAG Based on Hybrid Search (Lexical + Semantic)


## Setting
 - Auto Reload
 - path for utils

In [23]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
import sys, os
module_path = "../../.."
sys.path.append(os.path.abspath(module_path))

## 1. Bedrock Client 생성

In [25]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

### ---- ⚠️ Un-comment and edit the below lines as needed for your AWS setup ⚠️ ----
- os.environ["AWS_DEFAULT_REGION"] = "<REGION_NAME>"  # E.g. "us-east-1"
- os.environ["AWS_PROFILE"] = "<YOUR_PROFILE>"
- os.environ["BEDROCK_ASSUME_ROLE"] = "<YOUR_ROLE_ARN>"  # E.g. "arn:aws:..."
- os.environ["BEDROCK_ENDPOINT_URL"] = "<YOUR_ENDPOINT_URL>"  # E.g. "https://..."

In [26]:
boto3_bedrock = bedrock.get_bedrock_client(
    assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
    endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
    region=os.environ.get("AWS_DEFAULT_REGION", None),
)

print (colored("\n== FM lists ==", "green"))
pprint (bedrock_info.get_list_fm_models(verbose=False))

Create new client
  Using region: None
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-west-2.amazonaws.com)

== FM lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Claude-V2-1': 'anthropic.claude-v2:1',
 'Cohere-Embeddings-En': 'cohere.embed-english-v3',
 'Cohere-Embeddings-Multilingual': 'cohere.embed-multilingual-v3',
 'Command': 'cohere.command-text-v14',
 'Command-Light': 'cohere.command-light-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Llama2-13b-Chat': 'meta.llama2-13b-chat-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'amazon.titan-text-express-v1',
 'Titan-Text-G1-Light': 'amazon.titan-text-lite-v1'}


## 2. Titan Embedding 및 LLM 인 Claude-v2 모델 로딩

### LLM 로딩 (Claude-v2)

In [27]:
from langchain.llms.bedrock import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [28]:
llm_text = Bedrock(
    model_id=bedrock_info.get_model_id(model_name="Claude-V2-1"),
    client=boto3_bedrock,
    model_kwargs={
        "max_tokens_to_sample": 1024
    },
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

llm_text

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f6be5656110>, model_id='anthropic.claude-v2:1', model_kwargs={'max_tokens_to_sample': 1024}, streaming=True, callbacks=[<langchain_core.callbacks.streaming_stdout.StreamingStdOutCallbackHandler object at 0x7f6c460f0550>])

### Embedding 모델 선택

In [29]:
from utils.rag import KoSimCSERobertaContentHandler, SagemakerEndpointEmbeddingsJumpStart

In [30]:
def get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name=None):

    if is_bedrock_embeddings:
        # We will be using the Titan Embeddings Model to generate our Embeddings.
        from langchain.embeddings import BedrockEmbeddings
        llm_emb = BedrockEmbeddings(
            client=boto3_bedrock,
            model_id=bedrock_info.get_model_id(
                model_name="Titan-Embeddings-G1"
            )
        )
        print("Bedrock Embeddings Model Loaded")

    elif is_KoSimCSERobert:
        LLMEmbHandler = KoSimCSERobertaContentHandler()
        endpoint_name_emb = endpont_name
        llm_emb = SagemakerEndpointEmbeddingsJumpStart(
            endpoint_name=endpoint_name_emb,
            region_name=aws_region,
            content_handler=LLMEmbHandler,
        )
        print("KoSimCSERobert Embeddings Model Loaded")
    else:
        llm_emb = None
        print("No Embedding Model Selected")

    return llm_emb

#### [중요] is_KoSimCSERobert == True 일시에 endpoint_name 을 꼭 넣어 주세요.

In [31]:
is_bedrock_embeddings = True
is_KoSimCSERobert = False
aws_region = os.environ.get("AWS_DEFAULT_REGION", None)

##############################
# Parameters for is_KoSimCSERobert
##############################
if is_KoSimCSERobert: endpont_name = "<endpoint-name>"
else: endpont_name = None
##############################

llm_emb = get_embedding_model(is_bedrock_embeddings, is_KoSimCSERobert, aws_region, endpont_name)   

Bedrock Embeddings Model Loaded


## 3. LangChainOpenSearch VectorStore 정의
### 선수 조건
- 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 OpenSearch Index 가 생성이 되어 있어야 합니다.
#### [중요] 아래에 aws parameter store 에 아래 인증정보가 먼저 입력되어 있어야 합니다.
- 01_preprocess_docs/01_parameter_store_example.ipynb 참고

In [35]:
import boto3
from utils.ssm import parameter_store

In [36]:
region=boto3.Session().region_name
pm = parameter_store(region)

In [37]:
opensearch_domain_endpoint = pm.get_params(
    key="opensearch_domain_endpoint",
    enc=False
)

opensearch_user_id = pm.get_params(
    key="opensearch_user_id",
    enc=False
)

opensearch_user_password = pm.get_params(
    key="opensearch_user_password",
    enc=True
)

In [38]:
opensearch_domain_endpoint = opensearch_domain_endpoint
rag_user_name = opensearch_user_id
rag_user_password = opensearch_user_password

http_auth = (rag_user_name, rag_user_password) # Master username, Master password

### Index 이름 셋팅
- 이전 노트북 01_preprocess_docs/02_load_docs_opensearch.ipynb를 통해서 생성된 OpenSearch Index name 입력

In [39]:
#index_name = "genai-poc-knox-incremental-1024c-256o-v7"
index_name = "<your index>"
index_name = "rag-index-ko"

### OpenSearch Client 생성

In [40]:
from utils.opensearch import opensearch_utils

In [41]:
os_client = opensearch_utils.create_aws_opensearch_client(
    aws_region,
    opensearch_domain_endpoint,
    http_auth
)

## 4. Retriever based on Hybrid Search 정의
- LangChain에서 제공하는 **BaseRetriever** 클래스를 상속받아 **Custom Retriever**를 정의 할 수 있습니다.
- 본 샘플코드 에서는 **Hybrid Search based Retriever**를 **정의**합니다. 

OpenSearch Hybrid 는 아래와 같은 방식으로 작동합니다.
- (1) Sematic serch를 통해 각 document별 relevant score 산출
- (2) Lexical search를 통해 각 document별 relevant score 산출
- (3-1) Rank-fusion 방식이 "simple weighted" 일 경우
    - 산출된 score에 대한 normalization 수행
    - 전체 결과에서 가장 높은 스코어는 표준화 과정을 통하여 스코어가 1.0 이 됨.
- (3-2) Rank-fusion 방식이 "Reciprocal Rank Fusion (RRF)" 일 경우
    - Paper: https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
    - Desc: https://medium.com/@sowmiyajaganathan/hybrid-search-with-re-ranking-ff120c8a426d
    - **RRF의 경우 score가 아닌 ranking 정보를 활용, 때문에 score normalization이 필요 없음**
    - ![rrf.png](../../../10_advanced_question_answering/img/rrf.png)

RRF는 langchain에서 "Ensemble Retriever" 이름으로 api를 제공합니다. 
- https://python.langchain.com/docs/modules/data_connection/retrievers/ensemble

In [42]:
from utils.rag import OpenSearchHybridSearchRetriever

- 필터 설정 예시
- filter=[ <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your first keyword**"}}, <BR>
    　{"term": {"metadata.[**your_metadata_attribute_name**]": "**your second keyword**"}},<BR>
]

In [43]:
opensearch_hybrid_retriever = OpenSearchHybridSearchRetriever(
    # necessary
    os_client=os_client,
    index_name=index_name,
    llm_emb=llm_emb,
    llm_text=llm_text,

    # option for lexical
    minimum_should_match=0,
    filter=[],

    # option for rank fusion
    fusion_algorithm="RRF", # ["RRF", "simple_weighted"], rank fusion 방식 정의
    ensemble_weights=[.51, .49], # [for lexical, for semantic], Lexical, Semantic search 결과에 대한 최종 반영 비율 정의

    # option for async search
    async_mode=True,

    # option for output
    k=5, # 최종 Document 수 정의
    verbose=False,
)

### Retrieval example
- default search

In [44]:
from utils.rag import show_context_used

In [63]:
query = "새로운 AR 데모 모드 기능을 도입한 Knox Capture 버전은 무엇인가요?"

In [64]:
%%time
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print (f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)


==========  Results  ==========

1. question: 새로운 AR 데모 모드 기능을 도입한 Knox Capture 버전은 무엇인가요?
2. # documents: 5
3. Documents: 

-----------------------------------------------
1. Chunk: 938 Characters
-----------------------------------------------
.Knox 관리 포털에서 Knox 클라우드 서비스를 함께 사용하는 기존 Knox Guard 고객인 경우 기본적으로 탐색 창에서 Knox Guard가 숨겨집니다.서비스 표시/숨기기
페이지로 이동하여 다시 추가하세요.녹스가드 어드밴스드 녹스가드 23.03은 _녹스 가드 어드밴스드_라는 새로운 라이선스를 제공합니다. 이 라이선스는 기존 녹스 가드 기능 외에
부가가치 기능을 제공합니다.이번 릴리스에서는 디바이스에 최대 다섯 개의 앱을 설치할 수 있습니다. 이전 버전에서는 한 개만 설치할 수 있었습니다.API를 사용하여 삭제된 디바이스의
PIN 생성 Knox Guard 23.03은 삭제된 디바이스의 PIN을 생성할 수 있는 새로운 API를 제공하여 디바이스 사용자가 디바이스를 잠금 해제할 수 있도록
지원합니다.PIN 잠금 생성과 관련된 API 호출은 다음과 같습니다. GET /KCS/v1.1/kg/Devices/Deleted/getPin, GET
/KCS/v1.1/kg/Devices/Deleted/getPinbyChallenge 녹스 가드용 새로운 녹스 웹후크 알림 API 이전에는 업데이트된 디바이스 정보를 가져오려면 녹스
클라우드 API를 반복해서 호출해야 했습니다.시리얼 Knox Cloud API 호출을 대량으로 수행하면 때때로 성능이 느려질 수 있습니다.23.03 릴리스에는 KWN 구독 API에
Knox Guard 이벤트를 구독할 수 있는 Knox 웹후크 알림이 도입되었습니다.구독한 이벤트가 완료되면 KWN은 API의 출력을 다시

- update parameters

In [65]:
opensearch_hybrid_retriever.update_search_params(
    k=10,
    minimum_should_match=0,
    filter=[],
    #filter=[
    #    {"term": {"metadata.seq_num": "64"}},
    #],
    async_mode=True
)

In [66]:
query = "새로운 AR 데모 모드 기능을 도입한 Knox Capture 버전은 무엇인가요?"
search_hybrid_result = opensearch_hybrid_retriever.get_relevant_documents(query)

print("\n==========  Results  ==========\n")
print(f'1. question: {query}')
print(f'2. # documents: {len(search_hybrid_result)}')
print("3. Documents: \n")

show_context_used(search_hybrid_result)


==========  Results  ==========

1. question: 새로운 AR 데모 모드 기능을 도입한 Knox Capture 버전은 무엇인가요?
2. # documents: 10
3. Documents: 

-----------------------------------------------
1. Chunk: 938 Characters
-----------------------------------------------
.Knox 관리 포털에서 Knox 클라우드 서비스를 함께 사용하는 기존 Knox Guard 고객인 경우 기본적으로 탐색 창에서 Knox Guard가 숨겨집니다.서비스 표시/숨기기
페이지로 이동하여 다시 추가하세요.녹스가드 어드밴스드 녹스가드 23.03은 _녹스 가드 어드밴스드_라는 새로운 라이선스를 제공합니다. 이 라이선스는 기존 녹스 가드 기능 외에
부가가치 기능을 제공합니다.이번 릴리스에서는 디바이스에 최대 다섯 개의 앱을 설치할 수 있습니다. 이전 버전에서는 한 개만 설치할 수 있었습니다.API를 사용하여 삭제된 디바이스의
PIN 생성 Knox Guard 23.03은 삭제된 디바이스의 PIN을 생성할 수 있는 새로운 API를 제공하여 디바이스 사용자가 디바이스를 잠금 해제할 수 있도록
지원합니다.PIN 잠금 생성과 관련된 API 호출은 다음과 같습니다. GET /KCS/v1.1/kg/Devices/Deleted/getPin, GET
/KCS/v1.1/kg/Devices/Deleted/getPinbyChallenge 녹스 가드용 새로운 녹스 웹후크 알림 API 이전에는 업데이트된 디바이스 정보를 가져오려면 녹스
클라우드 API를 반복해서 호출해야 했습니다.시리얼 Knox Cloud API 호출을 대량으로 수행하면 때때로 성능이 느려질 수 있습니다.23.03 릴리스에는 KWN 구독 API에
Knox Guard 이벤트를 구독할 수 있는 Knox 웹후크 알림이 도입되었습니다.구독한 이벤트가 완료되면 KWN은 API의 출력을 다

## 5. RAG using RetrievalQA powered by LangChain

In [67]:
from utils.rag import prompt_repo
from utils.rag import run_RetrievalQA
from langchain.prompts import PromptTemplate

### Prompting
- [TIP] Prompt의 instruction의 경우 한글보다 영어로 했을 때 더 좋은 결과를 얻을 수 있습니다.

In [68]:
PROMPT = prompt_repo.get_qa(prompt_type="answer_only") # ["answer_only", "answer_with_ref"]
pprint (PROMPT.template)

('\n'
 '            \n'
 '\n'
 'Human:\n'
 '            You are a master answer bot designed to answer software '
 "developer's questions.\n"
 "            I'm going to give you a context. Read the context carefully, "
 "because I'm going to ask you a question about it.\n"
 '\n'
 '            Here is the context: <context>{context}</context>\n'
 '            \n'
 '            First, find a few paragraphs or sentences from the context that '
 'are most relevant to answering the question.\n'
 '            Then, answer the question as much as you can.\n'
 '\n'
 '            Skip the preamble and go straight into the answer.\n'
 "            Don't insert any XML tag such as <context> and </context> when "
 'answering.\n'
 '            \n'
 '            Here is the question: <question>{question}</question>\n'
 '\n'
 '            If the question cannot be answered by the context, say "No '
 'relevant context".\n'
 '            \n'
 '\n'
 'Assistant: Here is the answer. ')


### Update Search Params (Optional)

In [69]:
from langchain.chains import RetrievalQA

In [70]:
opensearch_hybrid_retriever.update_search_params(
    k=5,
    minimum_should_match=3,
    #filter=[
    #    {"term": {"metadata.project": "KS"}},
    #],
    filter=[],
    async_mode=True
)

### Request

In [71]:
qa = RetrievalQA.from_chain_type(
    llm=llm_text,
    chain_type="stuff",
    retriever=opensearch_hybrid_retriever,
    return_source_documents=True,
    chain_type_kwargs={
        "prompt": PROMPT,
        "verbose": False,
    },
    verbose=False
)

In [72]:
#query = "What version of Knox Capture introduced the new AR demo mode feature?"
query = "새로운 AR 데모 모드 기능을 도입한 Knox Capture 버전은 무엇인가요?"
response = qa(query)

 The Knox Capture version that introduces the new AR demo mode feature is 23.03.

In [73]:
print("##################################")
print("query: ", query)
print("##################################")

print (colored("\n\n### Answer ###", "blue"))
print_ww(response['result'])

print (colored("\n\n### Contexts ###", "green"))
show_context_used(response['source_documents'])

##################################
query:  새로운 AR 데모 모드 기능을 도입한 Knox Capture 버전은 무엇인가요?
##################################


### Answer ###
 The Knox Capture version that introduces the new AR demo mode feature is 23.03.


### Contexts ###
-----------------------------------------------
1. Chunk: 938 Characters
-----------------------------------------------
.Knox 관리 포털에서 Knox 클라우드 서비스를 함께 사용하는 기존 Knox Guard 고객인 경우 기본적으로 탐색 창에서 Knox Guard가 숨겨집니다.서비스 표시/숨기기
페이지로 이동하여 다시 추가하세요.녹스가드 어드밴스드 녹스가드 23.03은 _녹스 가드 어드밴스드_라는 새로운 라이선스를 제공합니다. 이 라이선스는 기존 녹스 가드 기능 외에
부가가치 기능을 제공합니다.이번 릴리스에서는 디바이스에 최대 다섯 개의 앱을 설치할 수 있습니다. 이전 버전에서는 한 개만 설치할 수 있었습니다.API를 사용하여 삭제된 디바이스의
PIN 생성 Knox Guard 23.03은 삭제된 디바이스의 PIN을 생성할 수 있는 새로운 API를 제공하여 디바이스 사용자가 디바이스를 잠금 해제할 수 있도록
지원합니다.PIN 잠금 생성과 관련된 API 호출은 다음과 같습니다. GET /KCS/v1.1/kg/Devices/Deleted/getPin, GET
/KCS/v1.1/kg/Devices/Deleted/getPinbyChallenge 녹스 가드용 새로운 녹스 웹후크 알림 API 이전에는 업데이트된 디바이스 정보를 가져오려면 녹스
클라우드 API를 반복해서 호출해야 했습니다.시리얼 Knox Cloud API 호출을 대량으로 수행하면 때때로 성